<h1 align="center"> A Primer on the LendingClub Loan Data </h1> <br>

I hope this kernel will help you build a broader perspective in observation for deeper exploration. This kernel also serve as a means of reflection, and to receive feedback from readers.

## Abstract

LendingClub is a US peer-to-peer lending company, headquartered in San Francisco, California. The data we are dealing with is a matrix of approximately 2 million observations with 145 variables capturing information on LendingClub’s issued loans from 2007 to 2019. 

## Motivation

Before we can perform any prediction or prescriptive modelling, we first try to read and understand the data with descriptive analysis. Descriptive analyses enable us to present the data in a more intuitive manner which allows for simpler interpretations. Specifically, descriptive analyses allow us to understand the distribution of the data or identify interesting patterns which may be relevant to our ultimate objective and aim we want to achieve with the data. 



In [ ]:
#imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import os
data_path = os.path.join('../input','loan.csv')

loan = pd.read_csv(data_path, low_memory=False)

#for predictive analysis
from sklearn.feature_selection import chi2
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

<h1 align="center"> Visual Inspection </h1> <br>

## Data Structure

The loans issued are indexed in time order, and across time, different loans are issued to different individuals. In other words, we are dealing with a repeated cross-sectional data.

In [ ]:
print(loan.info())

In [ ]:
loan.head(5)

Because repeated cross-sectional data comprise of different loans issued to different borrowers over time, it should be used to analyse changes in the loans issued or borrowers over time. It cannot be used to look at individual change. For example, we can draw conclusions on how the overall composition of ‘home_ownership’ of the borrowers has changed over time but we cannot deduce how the ‘home_ownership’ for a given borrower has changed over time. To study the aggregate change over time, we can:
* Construct panel at a more aggregated level
* Use time-series aspects to compare the different groups of loans/borrowers


## Missing Values

Missing values are non-trivial as columns or rows with many missing values are usually biased and thus, may be not representative of the whole dataset. About 33% of the entries are missing while 27% of the columns have more than 80% of missing values.


In [ ]:
missing_val_count_by_column = (loan.isnull().sum())
total_entries = len(loan)*len(loan.columns)
percent_missing_entries = sum(missing_val_count_by_column)/total_entries
percent_missing_columns = len(missing_val_count_by_column[missing_val_count_by_column > 0.8*len(loan)])/len(loan.columns)

print("Total missing entries:")
print(sum(missing_val_count_by_column),'\n')
print("Percentage of missing entries:")
print(percent_missing_entries*100, '\n')
print("Percentage of columns with more than 80 percent missing values:")
print(percent_missing_columns*100, '\n')
print("Columns with more than 80 percent missing values:")
print(missing_val_count_by_column[missing_val_count_by_column > 0.8*len(loan)])

The simplest option to handle missing values would be to remove columns or rows with missing values. A better option, however, would be to impute the missing values – using an aggregate measure or regression. A kernel on approaches to handle missing values can be found [here](https://www.kaggle.com/alexisbcook/missing-values).


<h1 align="center"> Descriptive Analysis </h1> <br>

## Aggregate Measures

Using mean and standard deviation as aggregate measures, we will look at some loan-specific variables to identify patterns in the data over time. We shall focus on loan amounts and interest rates of loans as a starting point.

### **Loan Amounts**

In [ ]:
loan['date_time'] = pd.to_datetime(loan['issue_d'])

In [ ]:
plt.figure(figsize=(14,5))

for x in ['loan_amnt','funded_amnt','funded_amnt_inv']:
    loan.groupby('date_time').mean()[x].plot(label=x)

plt.title('Mean of Loan Amounts')
plt.xlabel('Year')
plt.ylabel('Dollars')
plt.legend(loc='center left', bbox_to_anchor = (1,0.5))

Key observations:
* Overall, **mean loan amounts have increased across time**
* **Large variations between 2007 and 2009:** average loan amount first doubled from 2007 to 2008 then dropped by half in mid-2008 before increasing from mid-2008 onwards. This ‘unusual’ behaviour could be attributed to the subprime mortgage which coincides with the timeframe. 
* **The three curves converge after 2011:** this could be inferred as an increase in investor confidence, where borrowers were more likely to have received entire loan amounts demanded after 2011 as compared to before. 

### **Interest Rate (of loans)**

In [ ]:
plt.figure(figsize=(14,5))

loan.groupby('date_time').mean()['int_rate'].plot()

plt.title('Mean of Interest Rate (%)')
plt.xlabel('Year')
plt.ylabel('%')

The mean of interest rate appears to follow a head and shoulders pattern with a upper bound of approximately 13 percent before 2011 followed by a peak of 15 percent in 2013 and then decrease from 2013 to 2016. One possible interpretation would be considering interest rate on loans to be a proxy for the "price" for loans. After all, interest rate is the cost of loan for borrowers. 

> It is likely that interest rates remained low during the subprime mortage crisis from 2007 to 2009 and started to increase from 2011 as the economy began to recover.

In [ ]:
plt.figure(figsize=(14,5))

loan.groupby('date_time').std()['int_rate'].plot()

plt.title('Standard Deviation of Interest Rate (%)')
plt.xlabel('Year')
plt.ylabel('%')

Interest rate standard deviations have also been increasing overtime and this can be implied as an increase in the diversity of borrowers.

In [ ]:
plt.figure(figsize=(14,5))

loan.groupby('date_time').median()['int_rate'].plot()

plt.title('Median of Interest Rate (%)')
plt.xlabel('Year')
plt.ylabel('%')

As a control, we can plot the median of interest rate to check if there are extreme values which may have skewed the mean plot. In short, the median and mean plot appears to have similar shapes and scale.

## Loans Characteristics

### **Number of Loans**

In [ ]:
loan['year'] = loan['date_time'].dt.year

In [ ]:
plt.figure(figsize=(12,5))
loan.groupby('year').count()['loan_status'].plot()
plt.xlabel('Year')
plt.ylabel('Count')
plt.title('Number of Loans')

We observe an exponential increase in the number of loan contracts made out between 2008 and 2015, followed by a gentler increase before picking up again from 2017 to 2019. This clearly means that the dataset we are dealing with is skewed to the later part of the timeframe. 

> *The growth in number of loan contracts could be attributed to the growth path of the firm. For example, LC issued an IPO later in 2014 and that could be ascribed to the sharp increase in number of loan contracts made out between 2012 to 2015, if there was a certain KPI attached to these numbers. *

### **Loan Status**

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(x='year',hue='loan_status',data=loan)
plt.title('Loan Status')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

There is a large number of current loans dating back to 2014. In this sense, we are not able to identify the distribution of loan status since we do not know the outcome of these ongoing loans. 

> From 2007 to 2015, we can say with certain that the number of fully paid loans has increased during the period. However, we cannot extend this inference from 2015 onwards due to the sizable proportion of ongoing loans.



In [ ]:
print('Count of Loan Status:')
print(loan['loan_status'].value_counts(), '\n')
print('Proportion of Fully Paid Loans (amongst completed loans):')
print(loan['loan_status'].value_counts()['Fully Paid']/np.sum([loan['loan_status'] != 'Current'])*100)

We also observe that a majority (70%) of the completed loans, those that are not ‘current’, are fully paid. 

> As such, there is an imbalance in loan status amongst the completed loans. 

<h1 align="center"> Loan Status Analysis </h1> <br>

Here we identify variables of interest to be the loan grade of the borrower, home ownership and income, region, and finally, purpose of loan. We will study possible correlation between loan status and the variables of interest. 

> Dummy variables were assigned to the samples of completed loans as (1) a 'Fully Paid' loans and (0) ‘otherwise’.

In [ ]:
def fully_paid(x):
    if x == 'Fully Paid':
        return int(1)
    else:
        return int(0)

In [ ]:
completed_loan = loan.copy()
completed_loan = completed_loan[completed_loan['loan_status'] != 'Current']
completed_loan['fully_paid_dummy'] = completed_loan['loan_status'].apply(fully_paid)

### **Grade**

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot(x='year',hue='grade',data=completed_loan,hue_order=['A','B','C','D','E','F','G'])
plt.title('Grade')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
print(completed_loan.groupby('grade')['fully_paid_dummy', 'int_rate'].mean())

We understand that loan grades are formed based on a pre-assessment of the borrower, and as it turns out, borrowers with higher loan grading had the largest concentration of good outcomes. This implies that LendingClub’s assigned loan grade to their respective borrowers have been internally consistent. Interest rates are also seen to be lower in contracts made out to higher grading borrowers.

### **Income & Home Ownership**

In [ ]:
print('Summary Stats for Annual Income (amongst completed loans):')
print(completed_loan.describe()['annual_inc'])

In [ ]:
# Creating bins for income

p1 = np.nanpercentile(np.array(completed_loan['annual_inc']),25)
p2 = np.nanpercentile(np.array(completed_loan['annual_inc']),50)
p3 = np.nanpercentile(np.array(completed_loan['annual_inc']),75)


def income_bin(x):
    if x <= p1:
        return 'Low'
    elif x <= p2:
        return 'Middle_Low'
    elif x <= p3:
        return 'Middle_High'
    else:
        return 'High'

In [ ]:
completed_loan['income_group'] = completed_loan['annual_inc'].apply(income_bin)

In [ ]:
print(completed_loan.groupby(['income_group','home_ownership'])['fully_paid_dummy','int_rate'].mean().sort_values('fully_paid_dummy', ascending = False))

A decoupling of the interest rates is observed, where the middle-income group of borrowers with no home ownership had larger good loan outcomes but held higher interest rates. 
>It appears that home ownership seems to be a key variable in determining interest levels for a loan contract, and that is independent of probability of loan outcome.

In [ ]:
print(completed_loan.groupby('home_ownership')['fully_paid_dummy','int_rate'].mean().sort_values('int_rate', ascending = False))

Observe that "none" has the highest mean interest rate despite having the 2nd highest rate of good loan.****

### **Region**

In [ ]:
west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']

def main_region(x):
    if x in west:
        return 'West'
    elif x in south_west:
        return 'South West'
    elif x in south_east:
        return 'South East'
    elif x in mid_west:
        return 'Mid West'
    else:
        return 'North East'

In [ ]:
completed_loan['main_region'] = completed_loan['addr_state'].apply(main_region)

In [ ]:
print(completed_loan.groupby('main_region')['fully_paid_dummy', 'int_rate'].mean().sort_values('fully_paid_dummy',ascending = False))

There appears to be no discrepancies in the different regions across America to which loan have been made out to.

### **Purpose**

In [ ]:
print(completed_loan.groupby('purpose')['fully_paid_dummy', 'int_rate'].mean().sort_values('fully_paid_dummy',ascending = False))

It is also clear that purpose is correlated with the rate of a loan being fully paid. One surprising observation would be that educational loans are doing the worst. This could possibly be due to the fact that students usually do not command incomes.

### **Employment Length**

In [ ]:
def get_int(x):
    if x == '10+ years':
        return int(10)
    elif x == np.nan:
        return np.nan
    else: 
        tokens = x.split()
        for s in tokens:
            if s.isdigit():
                return int(s)
        

In [ ]:
completed_loan['emp_int'] = completed_loan['emp_length'].astype(str).apply(get_int)

print(completed_loan.groupby('purpose')['fully_paid_dummy', 'int_rate'].mean().sort_values('fully_paid_dummy',ascending = False))

<h1 align="center"> Predictive Analysis </h1> <br>

Let's try to fit a predictive model to our data. As a baseline model, we won't be putting much emphasis on feature engineering and feature selection. 

In [ ]:
# cleaning up data 

completed_loan['term int'] = completed_loan['term'].apply(get_int)


# remove columns with more than 10% missing values
cols_with_missing = [col for col in completed_loan.columns 
                                 if completed_loan[col].isnull().sum() > 0.1*len(completed_loan)]

reduced_df = completed_loan.drop(cols_with_missing, axis = 1)

# drop irrelevant columns
reduced_df.drop(['term','emp_length','emp_title','zip_code', 'addr_state','earliest_cr_line',
        'disbursement_method','year','date_time','total_rec_late_fee', 'policy_code','num_tl_120dpd_2m',
                'last_pymnt_d','last_credit_pull_d','loan_status','issue_d','title'],axis=1,inplace=True)

# encoded categorical variables
dummies = pd.get_dummies(reduced_df.select_dtypes(include='object'),drop_first=True)


# training data
train = pd.concat([reduced_df.select_dtypes(exclude='object'), dummies], axis = 1).dropna(axis=0)

In [ ]:
x_train = train.drop('fully_paid_dummy', axis = 1)

y = train['fully_paid_dummy']

In [ ]:
# Setup
n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, random_state=1, shuffle=True)
y_oof = y*0 
feat_impt = 0
model = RandomForestClassifier(n_estimators=50,random_state=0)

In [ ]:
print("\n Begin Setting up cv. Executing {} folds cross validation \n".format(n_folds))
for i, (train_index,test_index) in enumerate(skf.split(x_train,y)):
        
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = x_train.iloc[train_index], x_train.iloc[test_index]
    print("\n Starting: fold {}".format(i+1))
        
    # fit model
    model.fit(X_train,y_train)
    
    # predict
    oof_pred = model.predict(X_valid)
    print('Accuracy Score:',accuracy_score(y_valid,oof_pred), '\n')
    print('Classification Report:','\n', classification_report(y_valid,oof_pred))
    
    # save
    y_oof.iloc[test_index] = oof_pred
    feat_impt += feat_impt + model.feature_importances_

The variables fitted produced an average accuracy score of 99.76%.

### **Feature Importance**

In [ ]:
feat_impt = pd.DataFrame(feat_impt,index=x_train.columns)
feat_impt.columns = ['Feature Importance']
plt.figure(figsize=(15,8))
sns.barplot(x=feat_impt.sort_values(by='Feature Importance',ascending=False).head(10).index,
            y=feat_impt.sort_values(by='Feature Importance',ascending=False)['Feature Importance'].head(10),
            palette='viridis')
plt.tight_layout()

Amongst the top 10 predictors identified through RFC to hold higher importance in capturing loan outcomes, 4 of them were recovery-related features. It is further intuitive to mention that debt-recovery only occurs when the loan contract has already gone ‘bad’. However, we are much more interested in identifying a ‘bad’ loan before it turns ‘bad’. 
> Thus, the features within our dataset might not hold strong form metrics in terms of predicting loan outcomes ex ante. 


<h1 align="center"> Final Words </h1> <br>

## Causality vs Correlation

> A casual observer looking at a strong correlation between variables and the outcome of loan may be tempted to conclude that, for instance, educational loans are riskier because it is negatively correlated with the outcome of being a good loan. This, however, is an unwarranted conclusion because correlation is not a statement of causality. Because correlation only measures direction and magnitude of change between two variables, it could be the case where the two variables are directly unrelated but are affected by a third variable that links to one another. That being said, correlation is often helpful in the exploratory data analysis phase to help us identify variables of interests. 

## Possible Bias

> Given that each loan already has an ‘assigned grade’, it appears that LendingClub already has in place some form of screening before approving a loan request. It is likely that the dataset only comprises of borrowers that have already passed an initial “screening” by LC which introduces bias in the dataset. Thus, the use of dataset for deeper analyses would be more suited for monitoring approved loans – narrowing the scope of loans to monitor.